In [34]:
from fractional_differentiation import find_stat_series
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import itertools
import multiprocessing as mp

# Load Data


In [9]:
prices = pd.read_csv("../../data/prices_processed.csv", parse_dates=["date"])

/var/folders/3r/t8b8j58n2hl6cxp_9t3pf6880000gn/T/ipykernel_2655/3840172344.py:1: DtypeWarning: Columns (17,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  prices = pd.read_csv("../../data/prices_processed.csv", parse_dates=["date"])


In [3]:
prices.tail(3)

,permno,date,nameendt,shrcd,exchcd,siccd,ticker,comnam,tsymbol,naics,primexch,trdstat,secstat,permco,issuno,hexcd,hsiccd,dclrdt,dlamt,paydt,rcrddt,shrflg,hsicmg,hsicig,distcd,divamt,facpr,facshr,acperm,accomp,shrenddt,nwperm,dlprc,trtscd,nmsind,mmcnt,nsdinx,bidlo,askhi,prc,vol,ret,bid,ask,shrout,cfacpr,cfacshr,openprc,numtrd,retx,vwretd,vwretx,ewretd,ewretx,sprtrn,vol_36mo,vol_252d,vol_126d,vol_21d,ret_vol_adj,mktcap,prc_adj,naics_processed,ret_industry_tot,ret_industry_relative,MACD_index,rsi,in_sp500
3890644,93436,2023-12-27,NaN,11,3,9999,TSLA,TESLA INC,TSLA,336110.0,Q,A,R,53453,66252,3,9999,NaN,NaN,NaN,NaN,NaN,99.0,999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,257.51999,263.34,261.44000,106250779.0,0.018822,261.51999,261.56000,3178921.0,1.0,1.0,258.35001,1050000.0,0.018822,0.001995,0.001807,0.005276,0.004907,0.001430,0.037041,0.035013,0.030002,0.021262,0.537571,8.310971e+08,261.44000,33.0,0.106531,0.431040,4.915272,68.511971,1
3890645,93436,2023-12-28,NaN,11,3,9999,TSLA,TESLA INC,TSLA,336110.0,Q,A,R,53453,66252,3,9999,NaN,NaN,NaN,NaN,NaN,99.0,999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,252.71001,265.13,253.17999,113250680.0,-0.031594,253.08000,253.19000,3178921.0,1.0,1.0,263.66000,1170000.0,-0.031594,-0.000108,-0.000295,0.002042,0.001428,0.000370,0.037050,0.034286,0.030136,0.020625,-0.921474,8.048392e+08,253.17999,33.0,0.008914,-0.930388,5.076989,53.706654,1
3890646,93436,2023-12-29,NaN,11,3,9999,TSLA,TESLA INC,TSLA,336110.0,Q,A,R,53453,66252,3,9999,NaN,0.0,NaN,NaN,0.0,99.0,999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,6.0,54.0,70.0,247.42999,255.19,248.48000,100321201.0,-0.018564,248.48000,248.49001,3178921.0,1.0,1.0,255.10001,1060000.0,-0.018564,-0.004045,-0.004084,-0.007480,-0.007546,-0.002826,0.037057,0.034266,0.030147,0.020932,-0.541765,7.898983e+08,248.48000,33.0,-0.221914,-0.319851,5.070092,47.033612,1


# Find Non-Stationary Series


In [17]:
# initialize dataframe
# store 1 if the feature for that permno is non-stationary
features = [
    "prc_adj",
    "vol",
    "openprc",
    "askhi",
    "bidlo",
    "bid",
    "ask",
    "numtrd",
    "ret",
    "retx",
    "mktcap",
    "ret_industry_relative",
    "MACD_index",
    "rsi",
]

non_stat = pd.DataFrame(index=prices["permno"].unique(), columns=features)

non_stat

,prc_adj,vol,openprc,askhi,bidlo,bid,ask,numtrd,ret,retx,mktcap,ret_industry_relative,MACD_index,rsi
10078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
prices.sort_values(by=["permno", "date"], inplace=True)

In [30]:
from ipywidgets import IntProgress
from IPython.display import display

# just to display progress
f = IntProgress(min=0, max=np.prod(non_stat.shape))
display(f)

# iterate through all permnos and features
for permno, feature in itertools.product(non_stat.index, non_stat.columns):
    f.value += 1

    # select the data relative to the permno and feature
    mask = prices["permno"] == permno
    data = prices.loc[mask, feature].dropna()

    # handle features which are empty
    if data.empty:
        print(f"there is no data for {permno} - {feature}")
        continue

    # check if the series is stationary
    if adfuller(data, regression="ct")[1] > 0.01:
        non_stat.loc[permno, feature] = 1
    else:
        non_stat.loc[permno, feature] = 0

IntProgress(value=0, max=11200)

there is no data for 10137 - numtrd
there is no data for 10147 - numtrd
there is no data for 10225 - numtrd
there is no data for 10516 - numtrd
there is no data for 11308 - numtrd
there is no data for 11404 - numtrd
there is no data for 11533 - numtrd
there is no data for 11674 - numtrd
there is no data for 11703 - numtrd
there is no data for 11754 - numtrd
there is no data for 11762 - numtrd
there is no data for 11850 - numtrd
there is no data for 11891 - numtrd
there is no data for 11955 - numtrd
there is no data for 12052 - numtrd
there is no data for 12060 - numtrd
there is no data for 12062 - numtrd
there is no data for 12076 - numtrd
there is no data for 12140 - numtrd
there is no data for 12345 - numtrd
there is no data for 12369 - numtrd
there is no data for 12431 - numtrd
there is no data for 12448 - numtrd
there is no data for 12449 - numtrd
there is no data for 12456 - numtrd
there is no data for 12476 - numtrd
there is no data for 12490 - numtrd
there is no data for 12542 -

In [33]:
non_stat.sum(axis=0)

prc_adj                  772
vol                       53
openprc                  740
askhi                    744
bidlo                    734
bid                      738
ask                      740
numtrd                    35
ret                        0
retx                       0
mktcap                   776
ret_industry_relative      1
MACD_index                 9
rsi                        5
dtype: object

In [41]:
non_stat.to_csv("../../data/non_stat.csv")